# OKR Notebook

In [1]:
# Import libraries 
import pandas as pd
import hashlib
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px
import plotly.graph_objects as go
import random

# export images
import plotly.io as pio
import os
import shutil
pio.renderers.default = "svg"
pio.orca.config.executable = "/Users/okunoe01/miniconda3/bin/orca"

import json

# For connecting to google sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
from df2gspread import gspread2df as g2d

import subprocess

In [2]:
IMG_DIR = "images"

try:
    shutil.rmtree(IMG_DIR)
except Exception as e:
    pass

os.mkdir(IMG_DIR)

In [3]:
# Authenticate Google Sheets
scope = ['https://spreadsheets.google.com/feeds']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'key/service_account_GS.json',
    scope
)

gc = gspread.authorize(credentials)

In [4]:
spreadsheet_key = '1_9r4cU3sL08hiOn-QJ2U-gd_fGyYzcv5B2l5t94FTnM'
wks_name = "Form responses 1"

df = g2d.download(
    credentials=credentials, 
    wks_name=wks_name, 
    gfile=spreadsheet_key,
    col_names=True
)

In [5]:
# Aliasing everyone's emails

df['emailHash'] = df['What is your email?'].apply(hash)

email_hashes = dict(zip(df['What is your email?'], df.emailHash))
hashes_email = dict(zip(df.emailHash, df['What is your email?']))
hashes = list(hashes_email.keys())

In [6]:
alias_df = df.drop(columns=['What is your email?'])

In [7]:
def get_plots(df, user):
    cleaned_df = df.loc[df.emailHash == user].drop(columns=['Comment', "emailHash"])
    column_names = cleaned_df.columns.values.tolist()
    rows = cleaned_df.transpose().to_numpy().tolist()
    return dict(zip(column_names, rows))

In [36]:
colors = [
 'aqua',
 'aquamarine',
 'black',
 'blue',
 'blueviolet',
 'brown',
 'burlywood',
 'cadetblue',
 'chartreuse',
 'chocolate',
 'coral',
 'cornflowerblue',
 'crimson',
 'cyan',
 'darkblue',
 'darkcyan',
 'darkgoldenrod',
 'darkgray',
 'darkgrey',
 'darkgreen',
 'darkkhaki',
 'darkmagenta',
 'darkolivegreen',
 'darkorange',
 'darkorchid',
 'darkred',
 'darksalmon',
 'darkseagreen',
 'darkslateblue',
 'darkslategray',
 'darkslategrey',
 'darkturquoise',
 'darkviolet',
 'deeppink',
 'deepskyblue',
 'dimgray',
 'dimgrey',
 'dodgerblue',
 'firebrick',
 'forestgreen',
 'fuchsia',
 'gold',
 'goldenrod',
 'gray',
 'grey',
 'green',
 'greenyellow',
 'hotpink',
 'indianred',
 'indigo',
 'khaki',
 'lawngreen',
 'lime',
 'limegreen',
 'magenta',
 'maroon',
 'mediumaquamarine',
 'mediumblue',
 'mediumorchid',
 'mediumpurple',
 'mediumseagreen',
 'mediumslateblue',
 'mediumspringgreen',
 'mediumturquoise',
 'mediumvioletred',
 'midnightblue',
 'navy',
 'olive',
 'olivedrab',
 'orange',
 'orangered',
 'orchid',
 'peru',
 'pink',
 'plum',
 'purple',
 'red',
 'rosybrown',
 'royalblue',
 'saddlebrown',
 'salmon',
 'sandybrown',
 'seagreen',
 'sienna',
 'silver',
 'skyblue',
 'slateblue',
 'slategray',
 'slategrey',
 'springgreen',
 'steelblue',
 'tan',
 'teal',
 'tomato',
 'turquoise',
 'violet',
 'yellow',
 'yellowgreen']

line_style = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']

In [12]:
def select_figures(heading, figures):
    # Removed o1, o2, m list that contained the heading to filter by.
    
    if heading == "Timestamp":
        return None
    else:
        if heading in o1:
            return figures["o1"]
        elif heading in o2:
            return figures["o2"]
        else:
            return figures["m"]

In [35]:
def plot_scatter(figures, plots):
    for k, v in plots.items():
        name = k #[:50]
        fig = select_figures(k, figures)
        if not fig:
            continue
        line = dict(
            color=random.choice(colors),
            dash=random.choice(line_style),
            width=3
        )
        fig.add_trace(
            go.Scatter(
                x=plots['Timestamp'],
                y=v,
                name=name,
                line=line
            )
        )

In [9]:
def edit_layout(figures, h):
    for topic, figure in figures.items():
        figure.update_layout(
            title=h,
            legend_orientation="h",
            yaxis=dict(
                title="Score",
                range=[0.5,5.5]
            ),
            
            autosize=True,
        )
        figure.show()
        figure.write_image(f'images/{h}-{topic}.svg')


In [7]:
def output_okrs(alias_df, h, format_img="svg"):
    plots = get_plots(alias_df, h)
    figures = {topic: go.Figure() for topic in ["o1","o2", "m"]}
    plot_scatter(figures, plots)
    edit_layout(figures, h)

In [10]:
for h in hashes:
    output_okrs(alias_df, h)

NameError: name 'alias_df' is not defined

In [11]:
with open('email_hash.json', 'w') as outfile:
    json.dump(email_hashes, outfile)